# Primer TP de Simulacion

## Grupo 2


### Alumnos:

- Duzac, Emilia
- Belletti, Gabriel Ignacio
- Vazquez, Nicolaz Alberto
- Cohen, Martin

### Generador de numeros aleatorio elegido

MIXMAX

## Ejercicio 1

Implementar generador de numeros aleatorios.
Sobre el generador de números al azar elegido, se pide:
- Implementarlo en función de la especificación del mismo.
- Modificarlo (si aplica) para obtener números al azar entre 0 y 1
- Realizar los gráficos que se consideren adecuados para mostrar las distribuciones de números pseudoaleatorios generados en ambos casos (explicar que se busca interpretar con cada uno de los gráficos)

In [ ]:
import numpy as np

In [4]:
# Parameters defined in paper
N = 139 # Matrix and vector size
s = 0  # Magic number
p = 9223372036853751941 # Modulus

In [ ]:
def generate_mixmax_matrix(size, magic_number):
    
    # Initialize the MIXMAX matrix with ones
    matrix = np.ones((size, size), dtype=int)
    
    # Fill the matrix with the specified pattern
    for i in range(size):
        for j in range(size):
            if i == j and j > 0:
                matrix[i][j] = 2
            elif j < i and j > 0:
                matrix[i][j] = (i + 1) - (j - 1)

    matrix[2][1] += magic_number

    return matrix

mixmax_matrix = generate_mixmax_matrix(N, s)

print(mixmax_matrix)

In [ ]:
def generate_initial_vector(seed, size, period):

    # Pseudo random vector from seed
    a = 6364136223846793005
    c = 1

    random_numbers = []
    x = seed
    for _ in range(size):
        x = ((a * x + c) % period) / period
        random_numbers.append(x)
    return random_numbers

initial_vector = generate_initial_vector(42, N, p)

print(initial_vector)

In [7]:
class MixmaxRNG:

    def __init__(self, seed, size, magic_number, period):
        # crea la matriz
        self.matrix = generate_mixmax_matrix(size, magic_number)
        # crea el vector inicial (no se retornaran sus valores)
        self.vector = generate_initial_vector(seed, size, period)

    """generates a uniform pseudo random value between 0 and 1,
    you can add an interval (a, b) to return a value between a and b"""
    def generate_number(self, interval=(0,1)):
        # valor pseudo aleatorio
        result = self._generate_new_vector()[0]
        return self._to_interval(result, interval)

    """calculates a new vector of numbers and replaces the current one with the new one with mod 1"""
    def _generate_new_vector(self):
        # calculo el nuevo vector
        new_vector = np.matmul(self.matrix, self.vector)
        # actualizo el vector con mod 1
        self.vector = [i % 1 for i in new_vector]

    def _to_interval(self, number, interval):
        # encuentro la escala para multiplicar el valor
        scale = interval[1] - interval[0]
        # retorno el valor por la escala trasladando el 0 al minimo del intervalo
        return (number * scale) + interval[0]
        
rng = MixmaxRNG(27, N, s, p)

In [ ]:
print(rng.generate_number())
print(rng.generate_number())
print(rng.generate_number())
print(rng.generate_number())
print(rng.generate_number())
print(rng.generate_number())

## Ejercicio 2

En este ejercicio se busca aplicar distintos test estadísticos vistos en la materia para evaluar tanto el
comportamiento del generador de números pseudoaleatorios elegido como el comportamiento de otro generador
implementado por otro grupo. El grupo al cual le deben evaluar el generador se encuentra en la planilla de grupos
en Drive.
Para esta tarea se debe solicitar al equipo asignado uno o más lotes de números generados de los tamaños que se
necesiten. El equipo que haya generado los números debe incluir en la notebook entregada el código utilizado
para generar los mismos.


a) Sobre los generadores implementados en el ejercicio 1:
- Proponer, y realizar, al menos 3 tests para evaluar su comportamiento.
- Evaluar e interpretar los resultados de cada test para distintos tamaños de muestras.


b) Sobre el generador implementado por el grupo asignado:
- Proponer, y realizar, al menos 3 tests para evaluar su comportamiento.
- Evaluar e interpretar los resultados de cada test para distintos tamaños de muestras.
- Comparar los resultados obtenidos para este generados con los obtenidos en el punto a)

## Ejercicio 3

Un uso posible de los satélites es la captura de imágenes, como se expone en
el trabajo:
Chen, W., Mackin, S., & Palmer, P. (2006). Performance Modelling of Imaging
Service of Earth Observation Satellites with Two-dimensional Markov Chain.
57th International Astronautical Congress. 10.2514/6.IAC-06-B5.3.05
Utilizando el generador del ejercicio 1, y los datasets provistos junto a este
enunciado, se pide:
- Reproducir la generación de localizaciones de imágenes al azar,
mostrada en la figura 5: “Distribution of targets over the globe” de la
sección 4. Resultados. En este caso las imágenes se generan de forma
uniformemente distribuida dentro de los parámetros de latitud y
longitud [−60◦,60◦] [−150◦, 150◦] , respectivamente.
- Modificar el método para sólo generar localizaciones de imágenes
dentro del continente australiano (figuras 22 y 23 del dataset). De aplicar, en función del método elegido,
calcular el factor de rendimiento de forma empírica.

## Ejercicio 4

Utilizando el generador de números pseudoaleatorios implementado en el ejercicio 1, y el trabajo
Li, Xiaomeng & li, Xingang & Wang, Jiangfeng. (2012). Using Cellular Automata to Investigate Pedestrian Conflicts
with Vehicles in Crosswalk at Signalized Intersection. Discrete Dynamics in Nature and Society. 2012.
10.1155/2012/287502.
Resolver:
- Simular al menos 2 escenarios planteados en el trabajo y comparar sus resultados.
- Proponer al menos 2 modificaciones a los escenarios planteados para peatones, autos, semáforos y
arterias de circulación, simularlas y comparar sus resultados.
- Realizar los gráficos que se consideren necesarios para mostrar los resultados
- Seleccionar un escenario de los simulados y realizar una animación utilizando los datos simulados

### Descripcion del escenario planteado:

Se utiliza un modelo de autómata celular para simular el tráfico.

Idealización del sistema físico en el que se supone que tanto el espacio como el tiempo son discretos y cada una de las unidades que
interactúan puede tener solo un número finito de estados discretos.
En los autómatas celulares el espacio se divide en muchas unidades, mediante métodos in-erráticos, cuadrícula con ciertas formas.
Cada unidad en la cuadrícula in-errática se llama celda, que solo puede tomar su valor en el conjunto de estados discretos limitados.
Todas las células siguen las mismas reglas de funcionamiento y dinámica discreta en la naturaleza y la sociedad.

Actualizar según las reglas parciales fijadas.
Un gran número de células constituyen la evolución del sistema dinámico mediante la simple interacción.
Los componentes más básicos de los autómatas celulares incluyen cuatro partes: celdas, espacio de celda, vecinos y reglas.
Además, también se debe incluir el estado de las celdas.

El modelo de autómata celular en el artículo se desarrolla para simular un cruce de peatones en una intersección señalizada con un
ciclo de tiempo de señal peatonal fijo de $90 s$.
La fase de la señal para peatones está diseñada en seis clases,
que tienen una duración de tiempo verde de $25 s$, $30 s$, $35 s$, $40 s$, $45 s$ y $50 s$, correspondientes a
$65 s$, $60 s$, $55 s$, $50 s$, $45 s$ y $40 s$ de duración del tiempo rojo, respectivamente.
El segmento de la carretera que conecta con la intersección está diseñado como carriles de dos vías para seis vehículos y
el ancho de cada carril es de 3,5 metros.
El ancho del paso de peatones también está diseñado en 6 clases, que son $2,5 m$, $3 m$, $3,5 m$, $4 m$, $4,5 m$ y $5 m$ con una longitud de cruce de $21 m$.
Además, se han establecido dos áreas de espera para peatones a ambos lados del paso de peatones con capacidad para 100 peatones como máximo cada vez.
En este modelo, el paso de peatones se divide en celdas cuadradas de $0,5 × 0,5 m^2$.
Cada celda está ocupada por un peatón o vacía. Las velocidades iniciales asignadas al peatón pueden ser 2, 3, 4, 5 o 6 celdas por segundo,
lo que representa que sus velocidades reales son $1 m/s$, $1,5 m/s$, $2 m/s$, $2,5 m/s$, y $3 $m/s$, respectivamente.
Las proporciones de los cinco tipos de velocidades se basan en los resultados de la observación de campo,
que comparten 27,3%, 52%, 13,7%, 4,8% y 2,2% respectivamente. A través de la comparación de la velocidad asignada y
el número de celdas vacías frontales verticales, cada peatón actualiza su velocidad en cada paso de tiempo.
Se supone que cada vehículo ocupa 6 × 5 celdas y su velocidad en el cruce de peatones es de $5 m/s$.

Generalmente, existen dos tipos de condiciones de contorno en los modelos de simulación de autómatas celulares:
la condición de contorno periódica y la condición de contorno abierto.
Por razones tanto prácticas como teóricas, las dos condiciones de contorno son preferibles en diferentes situaciones.

- En la condición de frontera periódica, los límites relativos están vinculados y se genera un estado estacionario invariante de traducción en el sistema.
- Por otro lado, la condición de límite abierto concuerda con una cierta probabilidad cuando se inserta y retira un vehículo de la celda límite,
en la que la invariancia traslacional se rompe, pero aún se pueden esperar estados estacionarios con un perfil de densidad no trivial.

En comparación con la condición de frontera periódica, la condición de frontera abierta es más realista y coherente para el modelo del artículo.
Por tanto, se adopta la condición de frontera abierta.

### Reglas del escenario planteado:

#### Celdas:

Cada celda mide 0.5 metros cuadrados ($0.5m × 0.5m$).

- Segmento de carretera: cuenta con dos carriles, para 6 vehiculos.
 - Carriles: ancho 3,5 metros (7 celdas).
- Paso de peatones:
-

#### Señales:

Total de periodo de semaforo 90 s (solo verde o rojo).
Fase de la señal para peatones (en segundos):

| Verde | Rojo |
|-------|------|
| 25    | 65   |
| 30    | 60   |
| 40    | 55   |
| 45    | 45   |
| 50    | 40   |

#### Movimiento:

Hay tres leyes básicas de movimiento en los modelos de autómatas celulares para describir el comportamiento de los peatones:

- movimiento lateral: representa el comportamiento de cambio de trayectoria de un peatón, cuyo resultado es obtener espacio de aceleración y evitar una colisión frontal.
- movimiento lineal: considera la velocidad de preferencia personal y otros peatones cercanos para identificar el número de pasos para avanzar.
- evitación de colisiones: se refiere al comportamiento de cómo evitar colisiones frontales entre peatones que caminan cerca uno del otro en dirección opuesta.

En la evolución de este modelo, tanto los peatones como los vehículos deben cumplir con las reglas de sus movimientos

#### Reglas para peatones:

Arribo, (es una distribucion Poisson).
$P_k = \frac{(λ_p)^{k}}{k!} × e^{-λt} = 0$

Donde:
- $P_k$ denota la probabilidad de alcanzar k peatones durante el intervalo de conteo t.
- $λ_p$ denota la tasa media de llegada de peatones cápita/segundo.
- $t$ denota cada intervalo de tiempo contado.

En la evolución de este modelo, los peatones dan prioridad a la celda frontal como destino.
El peatón puede avanzar en las celdas o cambiar a la izquierda o a la derecha cuando está bloqueado.

La velocidad inicial del peatón que ingresa al cruce de peatones $v_{i,j}$, se asigna de acuerdo con:
- 6 caso: $p > 0.978$
- 5 caso: $0.978 ≥ p > 0.93$
- 4 caso: $0.93 ≥ p > 0.793$
- 3 caso: $0.793 ≥ p > 0.273$
- 2 caso: $0.273 ≥ p ≥ 0$

Donde:
- $p$ es un número aleatorio entre 0 y 1.
- $v_{i,j}$ denota la velocidad del peatón cuya ubicación es $x_{i,j}$.

La dinámica discreta en la naturaleza y la sociedad La velocidad del peatón en el cruce de peatones durante la simulación
se actualiza sobre la base de $v_{i,j} = min(d_{i,j}, v_{i,j})$.
Donde, $d_{i,j}$ denota el número de celdas vacías entre el peatón objetivo y su peatón verticalmente frontal más cercano.

In [ ]:
"""dado un valor aleatorio y dos vectores, uno de valores ordenado de mayor a menor y otro de resultados.
retornara el equivalente de los resultados segun el mayor numero del vector al que el valor aleatorio sea mayor."""
def get_discrete_value_according_to_probability(val_p, interval, vec):
    # caso nulo retorna el ultimo valor
    if val_p == 0.0:
        return vec[-1]
    # verifica que sea mayor al numero del vector
    # (el vector empieza ordenado como pre condicion)
    for i in range(len(interval)):
        if val_p > interval[i]:
            return vec[i]

"""given a random value it will return The initial speed of the pedestrian entering the crosswalk,
according to the previous rules."""
def pedestrian_initial_velocity(val_p):
    # valor de la velocidad segun las reglas
    vec = (6, 5, 4, 3, 2)
    # probabilidad del valor de la velocidad segun las reglas
    interval = (0.978, 0.93, 0.793, 0.273, 0.0)
    return get_discrete_value_according_to_probability(val_p, interval, vec)

"""Test of pedestrian_initial_velocity"""
def test_pedestrian_initial_velocity():
    val = pedestrian_initial_velocity(1.0) == 6
    val &= pedestrian_initial_velocity(0.979) == 6
    val &= pedestrian_initial_velocity(0.978) == 5
    val &= pedestrian_initial_velocity(0.931) == 5
    val &= pedestrian_initial_velocity(0.930) == 4
    val &= pedestrian_initial_velocity(0.794) == 4
    val &= pedestrian_initial_velocity(0.793) == 3
    val &= pedestrian_initial_velocity(0.274) == 3
    val &= pedestrian_initial_velocity(0.273) == 2
    val &= pedestrian_initial_velocity(0.001) == 2
    val &= pedestrian_initial_velocity(0.0) == 2
    if val:
        print("OK")
    else:
        print("ERROR")

test_pedestrian_initial_velocity()